In [43]:
#!pip install packagename
# importing modules
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib
import matplotlib.pyplot as plt
import os
from os import chdir as cd
import time
import fiona

In [44]:
US_places = gpd.read_file(r'D:\Work\Box Sync\Trends_all states\Maps_2020\compiled.shp')
US_counties = gpd.read_file(r'D:\Work\Box Sync\Trends_all states\Maps_2020\tl_2020_us_county.zip')
US_states = gpd.read_file(r'D:\Work\Box Sync\Trends_all states\Maps_2020\tl_2020_us_state.zip')

# source: https://gis.stackexchange.com/questions/141580/which-projection-is-best-for-mapping-the-contiguous-united-states
US_states = US_states.to_crs('EPSG:9311')
US_places = US_places.to_crs('EPSG:9311')
US_counties = US_counties.to_crs('EPSG:9311')

US_places.shape, US_counties.shape, US_states.shape

((32188, 17), (3234, 18), (56, 15))

In [45]:
US_places.groupby(['STATEFP'])['GEOID'].count().sum()

32188

In [62]:
from pathlib import Path
# define the file location   CT_scernario_2020_22_tractssp4_total_2050
scenario = ['ssp1', 'ssp2', 'ssp3', 'ssp4', 'ssp5']
folder = Path(r"D:\Work\Box Sync\Spatial_Population_geotiff\Population from SSP\r_extract_output\\")
for i, s in enumerate(scenario):
    print(s)
    # reading the zip file
    csv_filenames = scenario[i] + '_total_*.tif.csv'
    filenames = folder.glob(csv_filenames)
    
    # combining places for US into one file
    master_df = pd.DataFrame()
    for filename in filenames:
        # print(filename)
        df = pd.read_csv(filename, encoding='latin-1').add_suffix(str(filename)[-13:-8])
        master_df = pd.concat([master_df, df], axis =1)
        # master_df.append(pd.concat([pd.read_csv(filename, index_col = 0, dtype = {'GEOID': str})], axis =1).pipe(pd.DataFrame).add_suffix(str(filename)[-8:-4]))

    print(master_df['population_2020'].sum())
    # master_df.shape
    # master_df.isna().sum()
    # master_df[master_df['GEOID_2010'] != master_df['GEOID_2100']]

    master_df['GEOID'] = master_df['GEOID_2100'].astype(str).str.rjust(7,'0')
    master_df.drop(master_df.filter(regex='GEOID_|NAME_|NAMELSAD_20').columns, axis =1, inplace = True)

    # master_df[master_df['std_2020'] > 2500]
    master_df.to_csv(r'D:\Work\Box Sync\Spatial_Population_geotiff\Population from SSP\r_concat_SSP\\population_from_' + str(s) + '.csv')


ssp1
239515351.66600987
ssp2
238873196.472252
ssp3
234495987.376714
ssp4
237511094.644772
ssp5
243379092.7204964


In [ ]:
# merging all scenarios

In [98]:
from pathlib import Path
# define the file location   CT_scernario_2020_22_tractssp4_total_2050
folder = Path(r"D:\Work\Box Sync\Spatial_Population_geotiff\Population from SSP\r_concat_SSP\\")
ssp_filenames = 'population_from_*' + '.csv'
files = folder.glob(ssp_filenames)
dfs = [pd.read_csv(f, index_col = 0, dtype = {'GEOID': str},) for f in files]

# Source: https://stackoverflow.com/questions/48236438/merge-multiple-dataframe-with-specified-suffix
suffix = ('_ssp1', '_ssp2', '_ssp3', '_ssp4', '_ssp5')
for i in range(len(dfs)):
    dfs[i] = dfs[i].add_suffix(suffix[i])

from functools import reduce
def agg_df(dfList):
    temp=reduce(lambda left, right: pd.merge(left, right, 
                                             left_index=True, right_index=True, 
                                             how='outer'), dfList)
    return temp

df = agg_df(dfs)
df.shape
df.to_csv(r"D:\Work\Box Sync\Spatial_Population_geotiff\Population from SSP\r_concat_SSP\NCAR_agg_scenarios.csv")

In [106]:
df.filter(regex= 'population').sum().to_csv(r"D:\Work\Box Sync\Spatial_Population_geotiff\Population from SSP\r_concat_SSP\NCAR_agg_scenarios_totals.csv")